In [8]:
import os
import numpy as np
import pandas as pd

from beat_tracker import beatTracker, ellisBeatTracker
from mir_eval import beat

np.int = np.int64

In [2]:
ballroom_cat = os.listdir("BallroomData")

In [3]:
dir_dict = {}

for cat in ballroom_cat:
    if cat not in ['nada', 'allBallroomFiles'] :
        files = os.listdir(f"BallroomData/{cat}")
        dir_dict[cat] = files



In [45]:
def get_eval_metrics(ref_beats_trimmed, beats_trimmed, print_metrics=False):
    f_measure = beat.f_measure(ref_beats_trimmed, beats_trimmed, f_measure_threshold=0.07)
    cemgil_score, cemgil_max = beat.cemgil(ref_beats_trimmed, beats_trimmed)
    goto = beat.goto(ref_beats_trimmed, beats_trimmed)
    p_score = beat.p_score(ref_beats_trimmed, beats_trimmed)
    info_gain = beat.information_gain(ref_beats_trimmed, beats_trimmed)
    CMLc, CMLt, AMLc, AMlt = beat.continuity(ref_beats_trimmed, beats_trimmed)

    metrics = {
        "f_measure": f_measure,
        "cemgil_score": cemgil_score,
        "cemgil_max": cemgil_max,
        "goto": goto,
        "p_score": p_score,
        "info_gain": info_gain,
        "CMLc": CMLc,
        "CMLt": CMLt,
        "AMLc": AMLc,
        "AMlt": AMlt,
    }
    if print_metrics:
        print(f"Evaluation metrics:")
        print(f"F-measure (0.07 threshold) = {f_measure}")
        print(f"Cemgil's score: {cemgil_score}; Maximum Cemgil's score: {cemgil_max}")
        print(f"Goto's score: {goto}")
        print(f"P-score: {p_score}")
        print(f"Information Gain: {info_gain}")
        print(f"CMLc: {CMLc}; CMLt: {CMLt}; AMLc: {AMLc}; AMlt: {AMlt}")

    return metrics


def evaluate_beats(beats_trimmed, ref_beats_trimmed):

    beat.validate(ref_beats_trimmed, beats_trimmed)

    # Evaluate beat
    metrics_dict = get_eval_metrics(ref_beats_trimmed, beats_trimmed)
    return metrics_dict

    

In [46]:
dfs = []
# tmp_dict = {"Rumba-American": dir_dict['Rumba-American']}
# for cat, file_list in tmp_dict.items():

for cat, file_list in dir_dict.items():
    print(f"Beat tracking for category: {cat}")
    eval_metrics = []
    for filename in file_list:
        audio_filename = f"BallroomData/{cat}/{filename}"
        beats_filename = f"BallroomAnnotations-master/{filename[:-4]}.beats"

        with open(beats_filename) as my_file:
            testsite_array = my_file.readlines()
            ref_beats = np.array([float(val.split()[0]) for val in testsite_array])
            ref_beat_count = np.array([float(val.split()[1]) for val in testsite_array])
            ref_down_beats = ref_beats[ref_beat_count==1]
            meter = int(ref_beat_count.max())

        bt = ellisBeatTracker(metric=meter)
        beats, downbeats = bt(audio_filename)

        # Trim beats to 5 seconds and above
        beats_trimmed = beat.trim_beats(beats, min_beat_time=5.0)
        ref_beats_trimmed = beat.trim_beats(ref_beats, min_beat_time=5.0)
        downbeats_trimmed = beat.trim_beats(downbeats, min_beat_time=5.0)
        ref_down_beats_trimmed = beat.trim_beats(ref_down_beats, min_beat_time=5.0)

        metrics_dict = evaluate_beats(beats_trimmed, ref_beats_trimmed)
        downbeat_metrics_dict = evaluate_beats(downbeats_trimmed, ref_down_beats_trimmed)
        downbeat_metrics_dict = {f'{metric_name}_downbeat': val for metric_name, val in downbeat_metrics_dict.items()}
        metrics_dict.update(downbeat_metrics_dict)

        # Evaluate Tempo
        actual_tempo = (60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1])).mean()
        actual_tempo_std = (60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1])).std()
        metrics_dict.update({"actual_tempo": actual_tempo, 
                           "est_tempo": bt.tempo, 
                           "tempo_diff": bt.tempo - actual_tempo, 
                           "actual_tempo_std": actual_tempo_std,
                           "actual_vs_est_tempo_ratio": actual_tempo/bt.tempo,
                           "filename": filename,
                           "meter_downbeat": meter})

        # Add all metrics to list
        eval_metrics.append(metrics_dict)
    
    cat_metrics_df = pd.DataFrame(eval_metrics)
    cat_metrics_df['category'] = cat
    dfs.append(cat_metrics_df)


Beat tracking for category: Waltz
Beat tracking for category: VienneseWaltz
Beat tracking for category: ChaChaCha
Beat tracking for category: Rumba-American
Beat tracking for category: Jive
Beat tracking for category: Tango
Beat tracking for category: Samba
Beat tracking for category: Quickstep
Beat tracking for category: Rumba-Misc
Beat tracking for category: Rumba-International


In [47]:
results = pd.concat(dfs)

In [48]:
downbeat_cols = [col for col in results.columns if 'downbeat' in col]

In [49]:
# Average results for beat detection and tempo est
results.drop(columns=downbeat_cols + ["filename", 'category']).mean()

f_measure                      0.691739
cemgil_score                   0.616754
cemgil_max                     0.766748
goto                           0.351003
p_score                        0.628919
info_gain                      0.450553
CMLc                           0.338683
CMLt                           0.361343
AMLc                           0.733653
AMlt                           0.799406
actual_tempo                 129.877539
est_tempo                    182.212479
tempo_diff                    52.334940
actual_tempo_std               2.991051
actual_vs_est_tempo_ratio      0.731201
dtype: float64

In [50]:
# Average results for downbeat detection
results[downbeat_cols].mean()

f_measure_downbeat       0.290554
cemgil_score_downbeat    0.257330
cemgil_max_downbeat      0.365534
goto_downbeat            0.153295
p_score_downbeat         0.384788
info_gain_downbeat       0.637430
CMLc_downbeat            0.177989
CMLt_downbeat            0.178732
AMLc_downbeat            0.445822
AMlt_downbeat            0.451529
meter_downbeat           3.749284
dtype: float64

In [51]:
# Average results for downbeat detection, for each category

results.groupby("category")[downbeat_cols].mean()

,f_measure_downbeat,cemgil_score_downbeat,cemgil_max_downbeat,goto_downbeat,p_score_downbeat,info_gain_downbeat,CMLc_downbeat,CMLt_downbeat,AMLc_downbeat,AMlt_downbeat,meter_downbeat
category,,,,,,,,,,,
ChaChaCha,0.134180,0.125036,0.198292,0.072072,0.204396,0.743413,0.080438,0.080438,0.244115,0.246074,4.0
Jive,0.068118,0.062378,0.089179,0.116667,0.187714,0.805035,0.166287,0.167212,0.237304,0.244468,4.0
Quickstep,0.361380,0.308133,0.462969,0.329268,0.471522,0.557124,0.443642,0.449291,0.637350,0.643933,4.0
Rumba-American,0.271201,0.231234,0.300144,0.142857,0.525230,0.397142,0.224490,0.224490,0.280390,0.280390,4.0
Rumba-International,0.215745,0.197977,0.290990,0.000000,0.271667,0.678754,0.000000,0.000000,0.301744,0.301744,4.0
Rumba-Misc,0.284051,0.248393,0.368844,0.000000,0.334854,0.639435,0.000000,0.000000,0.416128,0.416128,4.0
Samba,0.291804,0.271407,0.401498,0.000000,0.382506,0.678212,0.000000,0.000000,0.418087,0.418087,4.0
Tango,0.209330,0.178972,0.387472,0.139535,0.346367,0.551598,0.165947,0.165947,0.423540,0.423540,4.0
VienneseWaltz,0.811901,0.740519,0.740963,0.800000,0.818249,0.745257,0.817065,0.817065,0.891527,0.910604,3.0


In [52]:
# Average results for beat detection and tempo estimation, for each category

results.drop(columns=downbeat_cols + ['filename']).groupby("category").mean()

,f_measure,cemgil_score,cemgil_max,goto,p_score,info_gain,CMLc,CMLt,AMLc,AMlt,actual_tempo,est_tempo,tempo_diff,actual_tempo_std,actual_vs_est_tempo_ratio
category,,,,,,,,,,,,,,,
ChaChaCha,0.744341,0.698917,0.900818,0.378378,0.655933,0.615170,0.376904,0.376904,0.905782,0.924745,122.303650,188.042909,65.739259,1.949749,0.717728
Jive,0.805505,0.744120,0.783073,0.800000,0.801633,0.596082,0.782899,0.787621,0.832258,0.845704,166.114688,167.777585,1.662897,3.021075,0.990305
Quickstep,0.800684,0.677446,0.691100,0.658537,0.775256,0.309473,0.588135,0.725622,0.588736,0.727579,204.292185,198.678329,-5.613856,5.252537,1.029239
Rumba-American,0.530732,0.435910,0.524309,0.142857,0.525581,0.221858,0.190700,0.230383,0.261645,0.384727,125.478501,180.688129,55.209628,2.708514,0.724243
Rumba-International,0.633239,0.575757,0.863047,0.000000,0.481913,0.533375,0.000000,0.000000,0.860725,0.913166,100.032235,196.746798,96.714563,1.894954,0.508678
Rumba-Misc,0.608150,0.530005,0.798144,0.000000,0.471751,0.442856,0.000000,0.000000,0.793465,0.874130,94.170421,186.866710,92.696289,1.927862,0.503550
Samba,0.578510,0.536050,0.799040,0.000000,0.450347,0.526200,0.000000,0.000000,0.805965,0.814548,100.334682,195.326529,94.991847,1.479404,0.514994
Tango,0.660283,0.569721,0.639434,0.465116,0.656711,0.345560,0.460095,0.476707,0.527455,0.570994,127.463714,159.533016,32.069302,3.327960,0.836288
VienneseWaltz,0.956947,0.870953,0.872921,0.923077,0.949657,0.495267,0.899389,0.938662,0.899389,0.939067,178.089268,177.667092,-0.422176,5.085680,1.002425


In [53]:
results.category.value_counts()

category
ChaChaCha              111
Waltz                  110
Tango                   86
Samba                   86
Quickstep               82
VienneseWaltz           65
Jive                    60
Rumba-International     51
Rumba-Misc              40
Rumba-American           7
Name: count, dtype: int64

In [2]:
audio_filename = "BallroomData/Quickstep/Albums-Ballroom_Classics4-20.wav"
beats_filename = "BallroomAnnotations-master/Albums-Ballroom_Classics4-20.beats"

In [3]:
beats = beatTracker(audio_filename)

audio size: 1401848, sampling rate: 44100
audio size after resampling: 254304
The estimated tempo is: 104.16666666666667 BPM
The estimated tempo (tps2 + tps3) is: 197.3684210526316 BPM
The period is: 76.0 samples per beat
Starting DP loop...
Starting backtrace...


In [8]:
with open(beats_filename) as my_file:
    testsite_array = my_file.readlines()
    ref_beats = np.array([float(val.split(" ")[0]) for val in testsite_array])

ref_beats_trimmed = beat.trim_beats(np.array(ref_beats), min_beat_time=5.0)
actual_tempo = 60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1]).mean()

print(f"The actual tempo is: {actual_tempo} BPM")

The actual tempo is: 202.8169014084507 BPM


In [9]:
beats_trimmed = beat.trim_beats(np.array(beats), min_beat_time=5.0)
beat.validate(ref_beats_trimmed, np.flip(beats_trimmed))

In [5]:
def evaluate_beat(ref_beats_trimmed, beats_trimmed, print_metrics=False):
    f_measure = beat.f_measure(ref_beats_trimmed, beats_trimmed, f_measure_threshold=0.07)
    cemgil_score, cemgil_max = beat.cemgil(ref_beats_trimmed, beats_trimmed)
    goto = beat.goto(ref_beats_trimmed, beats_trimmed)
    CMLc, CMLt, AMLc, AMlt = beat.continuity(ref_beats_trimmed, beats_trimmed)

    metrics = {
        "f_measure": f_measure,
        "cemgil_score": cemgil_score,
        "cemgil_max": cemgil_max,
        "goto": goto,
        "CMLc": CMLc,
        "CMLt": CMLt,
        "AMLc": AMLc,
        "AMlt": AMlt,
    }
    if print_metrics:
        print(f"Evaluation metrics:")
        print(f"F-measure (0.07 threshold) = {f_measure}")
        print(f"Cemgil's score: {cemgil_score}; Maximum Cemgil's score: {cemgil_max}")
        print(f"Goto's score: {goto}")
        print(f"CMLc: {CMLc}; CMLt: {CMLt}; AMLc: {AMLc}; AMlt: {AMlt}")

    return metrics

def evaluate_beats(ref_beats_all, beats_all):
    metrics_all = []
    for ref_beat, est_beat in zip(ref_beats_all, beats_all):
        metrics = evaluate_beat(ref_beat, np.flip(est_beat))
        metrics_all.append(metrics)

    df = pd.DataFrame(metrics_all)
    
    return df




In [21]:
evaluate_beat(ref_beats_trimmed, np.flip(beats_trimmed), print_metrics=True)

Evaluation metrics:
F-measure (0.07 threshold) = 0.8941176470588236
Cemgil's score: 0.7730132745338721; Maximum Cemgil's score: 0.7730132745338721
Goto's score: 1.0
CMLc: 0.8705882352941177; CMLt: 0.8705882352941177; AMLc: 0.8705882352941177; AMlt: 0.8705882352941177


{'f_measure': 0.8941176470588236,
 'cemgil_score': 0.7730132745338721,
 'cemgil_max': 0.7730132745338721,
 'goto': 1.0,
 'CMLc': 0.8705882352941177,
 'CMLt': 0.8705882352941177,
 'AMLc': 0.8705882352941177,
 'AMlt': 0.8705882352941177}

In [20]:

beat.p_score(ref_beats_trimmed, np.flip(beats_trimmed))

0.4942528735632184

In [19]:
np.int = np.int64

In [25]:
60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1]).std()

26551.334373548536

In [30]:
(60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1])).mean()

104.13383690286433

In [31]:
(60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1])).std()

0.40937699328700883

In [32]:
(60/(ref_beats_trimmed[1:] - ref_beats_trimmed[:-1])).var()

0.16758952263271168